In [1]:
import os
import sys 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns

from skimage import io
from skimage.util import view_as_windows
from keras.utils import np_utils
from keras.optimizers import SGD
from keras.callbacks import CSVLogger, ModelCheckpoint
from keras.layers import Dense
from PIL import Image, ImageDraw

sys.path.append('/home/tanuj/Workspace/power-grid-detection')

%matplotlib inline
Image.MAX_IMAGE_PIXELS = None

Using Theano backend.
Using gpu device 1: GeForce GTX 680 (CNMeM is disabled, cuDNN 5005)


In [2]:
import config

from utils.model.helpers import get_model_from_json
from utils.img.helpers import sliding_window
from utils.dataset.helpers import get_image_collection
from utils.img.collection import ImageCollection
from utils.dataset.data_generator import DataGenerator

/home/tanuj/.tools/anaconda3/envs/py27/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
lr = 0.01
batch_size = 32
batch_size=256
n_epochs=500
input_shape=(140, 140, 1)
name = 'cnn_140_1_thr_dil_ero_lr_%f_conv_freeze' % lr

In [4]:
model = get_model_from_json('cnn_140_1_thr_dil_ero_lr_0.100000_final.json')
model.load_weights('/home/tanuj/Workspace/power-grid-detection/training/cnn_140_1_thr_dil_ero_lr_0.100000_training_weights_best.hdf5')
model

In [5]:
for layer in model.layers:
    print(layer)
    if not isinstance(layer, Dense):
        layer.trainable = False

In [6]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 140, 140, 1)   0                                            
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 140, 140, 128) 0           input_1[0][0]                    
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 140, 140, 128) 0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 70, 70, 128)   0           activation_1[0][0]               
___________________________________________________________________________________________

In [7]:
optimizer = SGD(lr=lr)

print('compiling model...')
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
print('done.')

compiling model...
done.


In [8]:
csv_logger = CSVLogger('%s_training.log' % name)
best_model_checkpointer = ModelCheckpoint(filepath=("./%s_training_weights_best.hdf5" % name), verbose=1,
                                          save_best_only=True)

In [9]:
print('Initializing data generators...')
train_data_gen = DataGenerator(dataset_file=config.train_data_file, batch_size=batch_size)
validation_data_gen = DataGenerator(dataset_file=config.validation_data_file, batch_size=batch_size)
test_data_gen = DataGenerator(dataset_file=config.test_data_file, batch_size=batch_size)
print('done.')

Initializing data generators...
done.


In [10]:
print('Fitting model...')
history = model.fit_generator(train_data_gen,
                              nb_epoch=n_epochs,
                              samples_per_epoch=train_data_gen.n_batches * batch_size,
                              validation_data=validation_data_gen,
                              nb_val_samples=validation_data_gen.n_samples,
                              verbose=1,
                              callbacks=[csv_logger, best_model_checkpointer])
print('done.')

Fitting model...
Epoch 1/500


MemoryError: Error allocating 2569011200 bytes of device memory (out of memory).
Apply node that caused the error: GpuAllocEmpty(Shape_i{0}.0, Shape_i{3}.0, Elemwise{Composite{((((i0 + (i1 * (i2 // i1))) - i2) // i3) + i3)}}[(0, 0)].0, Elemwise{Composite{((((i0 + (i1 * (i2 // i1))) - i2) // i3) + i3)}}[(0, 0)].0)
Toposort index: 84
Inputs types: [TensorType(int64, scalar), TensorType(int64, scalar), TensorType(int64, scalar), TensorType(int64, scalar)]
Inputs shapes: [(), (), (), ()]
Inputs strides: [(), (), (), ()]
Inputs values: [array(256), array(128), array(140), array(140)]
Outputs clients: [[GpuDnnConv{algo='small', inplace=True}(GpuContiguous.0, GpuContiguous.0, GpuAllocEmpty.0, GpuDnnConvDesc{border_mode='half', subsample=(1, 1), conv_mode='conv', precision='float32'}.0, Constant{1.0}, Constant{0.0})]]

Debugprint of the apply node: 
GpuAllocEmpty [id A] <CudaNdarrayType(float32, 4D)> ''   
 |Shape_i{0} [id B] <TensorType(int64, scalar)> ''   
 | |input_1 [id C] <TensorType(flo

In [ ]:
print('Evaluating model...')
score = model.evaluate_generator(test_data_gen, val_samples=test_data_gen.n_samples)
print('done.')

print('Test score:', score[0])
print('Test accuracy:', score[1])